In [1]:
import scanpy as sc
import squidpy as sq
import os

/home/augusta/anaconda3/envs/scanpy_env/lib/python3.10/site-packages/dask/dataframe/__init__.py:31: FutureWarning: The legacy Dask DataFrame implementation is deprecated and will be removed in a future version. Set the configuration option `dataframe.query-planning` to `True` or None to enable the new Dask Dataframe implementation and silence this warning.
  warnings.warn(


In [2]:
# import sys
# sys.path.append('/home/augusta/SSS_mount/insituCNV/InSituCNV/modules')
# sys.path

# Functions

In [3]:
import random
import pandas as pd
import numpy as np
import anndata

## generate_cnvs

In [4]:
def generate_cnvs(CNV_dict, min_size, max_size, gene_info, save_csv = None):
    """ 
    This function generates copy number variations (CNVs) based on a dictionary (CNV_dict) of genes (keys) and whether they should be gain or loss (value). The size (bp) of the CNV is a randomly chosen between min_size and max_size. The gene will be in the center of the CNV, so half of the size (bp) is subtracted (start) and added (end) from the center of the gene position ('Gene end (bp)'- 'Gene start (bp)')/2), according to the gene_list. The function returns the CNV information as a DataFrame, specifying the gene name, chromosome, size (bp), type (gain/loss), start (bp), end (bp).

    Parameters:
        - CNV_dict (dictionary): Gene name (key) and whether they should be 'gain' or 'loss'.
        - min_size (nbr): minimum size (in bp) of the CNV.
        - max_size (nbr): maximum size (in bp) of the CNV.
        - gene_info (str): The path to the gene information from the *Ensmbl_BioMart_gene_info.txt* file, including  'Gene stable ID', 'Chromosome/scaffold name', 'Gene start (bp)', 'Gene end (bp)', 'Gene name' of the human genome GRCh38.

    Returns:
        - cnv_df (DataFrame): Compiling the gene name, chromosome, size, type, start, end
    """

    gene_info = pd.read_csv(gene_info)
    cnv_list = []

    for gene in CNV_dict:
        # Check if the gene is present in gene_info DataFrame
        if gene not in gene_info['Gene name'].values:
            print(f"Gene '{gene}' not found in gene_info DataFrame.")
            continue
        
        # Find gene details in the gene_info DataFrame
        gene_row = gene_info.loc[gene_info['Gene name'] == gene].iloc[0]
        
        # Calculate gene center
        gene_center = (gene_row['Gene end (bp)'] + gene_row['Gene start (bp)']) // 2
        
        # Randomly generate CNV size
        cnv_size = random.randint(min_size, max_size)
        
        # Calculate CNV start and end positions
        cnv_start = gene_center - cnv_size // 2
        cnv_end = gene_center + cnv_size // 2
        
        # Create CNV entry
        cnv = {
            'Gene name': gene,
            'Chromosome': gene_row['Chromosome/scaffold name'],
            'Size (bp)': cnv_size,
            'Type': CNV_dict[gene],
            'Start (bp)': cnv_start,
            'End (bp)': cnv_end
        }
        
        cnv_list.append(cnv)
    
    # Convert the list of CNV dictionaries to a DataFrame
    cnv_df = pd.DataFrame(cnv_list)

    if save_csv:
        cnv_df.to_csv(save_csv, index=None)
    
    return cnv_df

## create_cnv_template

In [5]:
import numpy as np
import pandas as pd

def create_cnv_template(adata, CNV_df):
    """ 
    This function generates a template for CNVs to an AnnData object.

    Parameters:
        - adata (AnnData): The AnnData object to which the CNVs should fit.
        - CNV_df (DataFrame): CNVs generated by the generate_cnvs function. 

    Returns:
        - cnv_template_df (DataFrame): A CNV template where each CNV (gene name) has a -1/0/1 value for each gene in the adata.var.
    """
    
    # Step 1: Order adata.var by chromosome and start position
    adata.var.sort_values(by=['chromosome', 'start'], inplace=True)
    
    # Initialize the CNV template matrix with zeros
    cnv_template = np.zeros((len(CNV_df), len(adata.var_names)))

    # Create a mapping of gene names to indices in the sorted AnnData object
    gene_to_index = {gene: idx for idx, gene in enumerate(adata.var_names)}

    # Step 2: Loop through each CNV in the CNV_df
    for i, row in CNV_df.iterrows():
        chromosome = row['Chromosome']
        cnv_start = row['Start (bp)']
        cnv_end = row['End (bp)']
        cnv_type = row['Type']
        
        # Find the genes in the corresponding chromosome that fall within the CNV region
        selected_genes = adata.var[
            (adata.var['chromosome'] == f"chr{chromosome}") &
            (adata.var['start'] >= cnv_start) &
            (adata.var['start'] <= cnv_end)
        ]
        
        # Determine the CNV effect: -1 for loss, +1 for gain
        cnv_effect = -1 if cnv_type == 'loss' else 1
        
        # Step 3: Set the corresponding entries in the CNV template matrix
        for gene in selected_genes.index:
            gene_index = gene_to_index[gene]
            cnv_template[i, gene_index] = cnv_effect
    
    # Convert the CNV template matrix to a DataFrame for easier interpretation
    cnv_template_df = pd.DataFrame(cnv_template, columns=adata.var_names)
    
    # Optionally, add the name of the CNV (gene name) as rownames in the DataFrame for reference
    if 'Gene name' in CNV_df.columns:
        cnv_template_df.index = CNV_df['Gene name'].values
    else:
        print("Warning: 'Gene name' column missing from CNV_df.")

    return cnv_template_df


## simulate_cnvs

#### version 1 (np.random.poisson 0.5 and 1.5)

In [25]:
def simulate_cnvs(adata, cnv_template_df, subclone_dict, cell_type_reference, cell_type_cnv):
    """ 
    Simulate CNVs in an AnnData object using Poisson sampling.

    Parameters:
        - adata (AnnData): The AnnData object where the CNVs should be simulated.
        - cnv_template_df (DataFrame): Templates for each CNV generated for the adata.
        - subclone_dict (dict): Specifies the subclones and their CNVs.
        - cell_type_reference (str): Column name in adata.obs indicating cell types.
        - cell_type_cnv (str or list): Specific cell types for which CNVs should be simulated.

    Returns:
        - AnnData: The updated AnnData object with simulated CNVs.
    """
    adata.layers['CNV_simulated'] = adata.layers['counts'].copy()
    adata.layers['CNV_GT'] = np.zeros(adata.shape)

    if isinstance(cell_type_cnv, str):
        cell_type_cnv = [cell_type_cnv]

    num_cells = adata[adata.obs[cell_type_reference].isin(cell_type_cnv)].shape[0]
    subclone_names = list(subclone_dict.keys())

    adata.obs.loc[adata.obs[cell_type_reference].isin(cell_type_cnv), 'simulated_subclone'] = np.random.choice(
        subclone_names, size=num_cells, replace=True
    )
    adata.obs.loc[~adata.obs[cell_type_reference].isin(cell_type_cnv), 'simulated_subclone'] = "N"

    for subclone, cnvs in subclone_dict.items():
        if not cnvs:
            continue

        subclone_cells = adata.obs['simulated_subclone'] == subclone

        for cnv in cnvs:
            if cnv not in cnv_template_df.index:
                raise ValueError(f"CNV {cnv} not found in cnv_template_df")

            cnv_row = cnv_template_df.loc[cnv]
            cnv_effects = cnv_row.values.flatten()

            for gene_idx, effect in enumerate(cnv_effects):
                values_to_modify = adata.layers['CNV_simulated'][subclone_cells, gene_idx].toarray().flatten()

                if effect == -1:
                    simulated_values = np.random.poisson(0.5 * values_to_modify)
                    adata.layers['CNV_GT'][subclone_cells, gene_idx] = -1
                elif effect == 1:
                    simulated_values = np.random.poisson(1.5 * values_to_modify)
                    adata.layers['CNV_GT'][subclone_cells, gene_idx] = 1
                else:
                    continue

                simulated_values = np.clip(simulated_values, 0, None).reshape(-1, 1)
                adata.layers['CNV_simulated'][subclone_cells, gene_idx] = simulated_values

    return adata


#### version 2 (multiplying by rho; random value between 0 and alfa)

In [6]:
def simulate_cnvs(adata, cnv_template_df, subclone_dict, cell_type_reference, cell_type_cnv, alpha=2):
    """ 
    Simulates CNVs by uniformly scaling gene expression values to mimic gains and losses.

    Parameters:
        - adata (AnnData): The AnnData object where the CNVs should be simulated.
        - cnv_template_df (DataFrame): Templates for each CNV generated for the adata.
        - subclone_dict (dictionary): Specifies the subclones and their CNVs.
        - cell_type_reference (str): Column name in adata.obs indicating cell types.
        - cell_type_cnv (str or list): Specific cell types in which CNVs should be simulated.
        - alpha (float): Maximum amplitude for scaling.

    Returns:
        - adata (AnnData): Updated AnnData with simulated CNV data in a new layer.
    """
    adata.layers['CNV_simulated'] = adata.layers['counts'].copy()
    adata.layers['CNV_GT'] = np.zeros(adata.X.shape)

    if isinstance(cell_type_cnv, str):
        cell_type_cnv = [cell_type_cnv]

    num_cells = adata[adata.obs[cell_type_reference].isin(cell_type_cnv)].shape[0]
    subclone_names = list(subclone_dict.keys())

    adata.obs.loc[adata.obs[cell_type_reference].isin(cell_type_cnv), 'simulated_subclone'] = np.random.choice(subclone_names, size=num_cells, replace=True)
    adata.obs.loc[~adata.obs[cell_type_reference].isin(cell_type_cnv), 'simulated_subclone'] = "N"

    for subclone, cnvs in subclone_dict.items():
        if not cnvs:
            continue

        subclone_cells = adata.obs['simulated_subclone'] == subclone

        for cnv in cnvs:
            cnv_row = cnv_template_df.loc[cnv]
            cnv_effects = cnv_row.values.flatten()

            for gene_idx, effect in enumerate(cnv_effects):
                values_to_modify = adata.layers['CNV_simulated'][subclone_cells, gene_idx].toarray().flatten()
                
                if effect == -1:
                    rho = np.random.uniform(0, alpha)
                    modified_values = values_to_modify / (1 + rho)
                    adata.layers['CNV_GT'][subclone_cells, gene_idx] = -1
                elif effect == 1:
                    rho = np.random.uniform(0, alpha)
                    modified_values = values_to_modify * (1 + rho)
                    adata.layers['CNV_GT'][subclone_cells, gene_idx] = 1
                else:
                    continue

                modified_values = np.clip(modified_values, 0, None).reshape(-1, 1)
                adata.layers['CNV_simulated'][subclone_cells, gene_idx] = modified_values

    return adata


In [ ]:
from scipy.sparse import lil_matrix, csr_matrix
import numpy as np

def simulate_cnvs(adata, cnv_template_df, subclone_dict, cell_type_reference, cell_type_cnv, alpha=2):
    """ 
    Simulates CNVs by uniformly scaling gene expression values to mimic gains and losses.

    Parameters:
        - adata (AnnData): The AnnData object where the CNVs should be simulated.
        - cnv_template_df (DataFrame): Templates for each CNV generated for the adata.
        - subclone_dict (dictionary): Specifies the subclones and their CNVs.
        - cell_type_reference (str): Column name in adata.obs indicating cell types.
        - cell_type_cnv (str or list): Specific cell types in which CNVs should be simulated.
        - alpha (float): Maximum amplitude for scaling.

    Returns:
        - adata (AnnData): Updated AnnData with simulated CNV data in a new layer.
    """
    # Prepare layers
    adata.layers['CNV_simulated'] = lil_matrix(adata.layers['counts'])
    adata.layers['CNV_GT'] = np.zeros(adata.X.shape)

    if isinstance(cell_type_cnv, str):
        cell_type_cnv = [cell_type_cnv]

    # Assign subclone labels
    cell_type_mask = adata.obs[cell_type_reference].isin(cell_type_cnv)
    num_cells = np.sum(cell_type_mask)
    subclone_names = list(subclone_dict.keys())

    adata.obs.loc[cell_type_mask, 'simulated_subclone'] = np.random.choice(
        subclone_names, size=num_cells, replace=True
    )
    adata.obs.loc[~cell_type_mask, 'simulated_subclone'] = "N"

    # Process subclones and apply CNV effects
    for subclone, cnvs in subclone_dict.items():
        if not cnvs:
            continue

        subclone_cells = (adata.obs['simulated_subclone'] == subclone).values
        if not np.any(subclone_cells):
            continue

        for cnv in cnvs:
            cnv_row = cnv_template_df.loc[cnv].values.flatten()

            for gene_idx, effect in enumerate(cnv_row):
                if effect == 0:
                    continue

                values_to_modify = adata.layers['CNV_simulated'][subclone_cells, gene_idx].toarray().flatten()

                rho = np.random.uniform(0, alpha)
                if effect == -1:
                    modified_values = values_to_modify / (1 + rho)
                    adata.layers['CNV_GT'][subclone_cells, gene_idx] = -1
                elif effect == 1:
                    modified_values = values_to_modify * (1 + rho)
                    adata.layers['CNV_GT'][subclone_cells, gene_idx] = 1

                # Clip and reshape
                modified_values = np.clip(modified_values, 0, None)
                adata.layers['CNV_simulated'][subclone_cells, gene_idx] = modified_values.reshape(-1, 1)

    # Convert back to CSR format for optimized downstream use
    adata.layers['CNV_simulated'] = csr_matrix(adata.layers['CNV_simulated'])

    return adata


#### version 3

In [33]:
import numpy as np
import pandas as pd

def simulate_cnvs(adata, cnv_template_df, subclone_dict, cell_type_reference, cell_type_cnv):
    """
    Simulates CNVs with uniform scaling for gains and losses, mimicking one-arm chromosomal alterations.

    Parameters:
        - adata (AnnData): The AnnData object where the CNVs should be simulated.
        - cnv_template_df (DataFrame): Templates for each CNV generated for the adata.
        - subclone_dict (dictionary): Specifies the subclones and their CNVs.
        - cell_type_reference (str): Column in adata.obs that defines cell type classification.
        - cell_type_cnv (str or list): The cell types for which CNVs should be simulated.

    Returns:
        - adata (AnnData): The updated AnnData object containing a 'CNV_simulated' layer with simulated counts.
    """
    # Create a new layer in adata to store the simulated CNV data
    adata.layers['CNV_simulated'] = adata.layers['counts'].copy()

    # Randomly assign cells to subclones based on subclone_dict
    if isinstance(cell_type_cnv, str):
        cell_type_cnv = [cell_type_cnv]  # Convert string to a list

    num_cells = adata[adata.obs[cell_type_reference].isin(cell_type_cnv)].shape[0]
    subclone_names = list(subclone_dict.keys())

    # Assign random subclone to cells of the desired cell type
    adata.obs.loc[adata.obs[cell_type_reference].isin(cell_type_cnv), 'simulated_subclone'] = np.random.choice(subclone_names, size=num_cells, replace=True)
    # Assign 'N' to cells that are not in the selected cell type
    adata.obs.loc[~adata.obs[cell_type_reference].isin(cell_type_cnv), 'simulated_subclone'] = "N"

    # Define scaling factors for gains and losses
    gain_factor = 1.5
    loss_factor = 0.5
    noise_std = 0.1  # Standard deviation of the Gaussian noise

    # Apply CNVs to the expression data based on the assigned subclone
    for subclone, cnvs in subclone_dict.items():
        if not cnvs:
            continue  # Skip if the CNV list is empty

        # Get cells assigned to the current subclone
        subclone_cells = adata.obs['simulated_subclone'] == subclone

        for cnv in cnvs:
            # Extract the CNV effect for this gene from cnv_template_df
            cnv_row = cnv_template_df.loc[cnv]
            cnv_effects = cnv_row.values.flatten()

            for gene_idx, effect in enumerate(cnv_effects):
                if effect == -1:  # Simulate loss
                    simulated_values = (
                        adata.layers['CNV_simulated'][subclone_cells, gene_idx].toarray().flatten() * loss_factor +
                        np.random.normal(0, noise_std, size=np.sum(subclone_cells))
                    )
                elif effect == 1:  # Simulate gain
                    simulated_values = (
                        adata.layers['CNV_simulated'][subclone_cells, gene_idx].toarray().flatten() * gain_factor +
                        np.random.normal(0, noise_std, size=np.sum(subclone_cells))
                    )
                else:
                    continue

                # Ensure non-negative values and maintain structure
                simulated_values = np.clip(simulated_values, 0, None).reshape(-1, 1)
                adata.layers['CNV_simulated'][subclone_cells, gene_idx] = simulated_values

    return adata


# Load AnnData

In [8]:
adata_path = "/home/augusta/SSS_mount/insituCNV/data/lung_organoids.h5ad"
adata = sc.read_h5ad(adata_path)

In [9]:
adata.obs.cell_type

N3_O1_AAACCCAAGCGTCAAG-1       secretory cell
N3_O1_AAACCCACACTAGGTT-1       secretory cell
N3_O1_AAACCCACAGTTGTCA-1           basal cell
N3_O1_AAACGAATCCAGTACA-1       secretory cell
N3_O1_AAACGCTTCTGGGCGT-1           basal cell
                                    ...      
N3_ALIEX_TTTGATCTCAATCGGT-1    secretory cell
N3_ALIEX_TTTGGAGCAAGCAGGT-1     ciliated cell
N3_ALIEX_TTTGGTTGTATACCCA-1    secretory cell
N3_ALIEX_TTTGGTTTCTGAATGC-1    secretory cell
N3_ALIEX_TTTGTTGGTCGCATGC-1    secretory cell
Name: cell_type, Length: 8892, dtype: category
Categories (3, object): ['ciliated cell', 'secretory cell', 'basal cell']

# Simulate the data to contain CNVs

Divide the dataset into four simulated subclones

1.   Normal (unaltered)
2.   Subclone A (fewer CNVs)
3.   Subclone B (same as A but added)
4.   Subclone C (same as A but added)


Choose chromosomal regions to be duplicated or deleted

If we generate several CNVs throughout the genome, it might be possible to compare the detection efficiency as to how many of the CNVs are detected. Or else, it's works/not works.


Things to consider: 
- how the size (nbr of cells) of the subclone affect the outcome - or make same size populations and avoid adressing this
- choose the size of the CNVs
    - literature to find an appropriate size for CNVs (50bp - several Mbs, ref: https://doi.org/10.1016%2Fj.bj.2021.02.003)
    - different sizes to see how that affects the outcome - or make every CNV the same size


## Module to simulate CNVs in adata (simulate_CNVs.py)

In [8]:
# from simulate_CNVs import *

### Function: Generate CNVs (generate_cnvs)

This function generates copy number variations (CNVs) based on a dictionary (**CNV_dict**) of genes (keys) and whether they should be gain or loss (value). The size (bp) of the CNV is a randomly chosen size between **min_size** and **max_size**. The gene will be in the center of the CNV, so half of the size (bp) is subtracted (start) and added (end) from the center of the gene position ('Gene end (bp)'- 'Gene start (bp)')/2), taken from the **gene_list**. The output will be a list of these CNVs, specifying the gene name, chromosome, size (bp), type (gain/loss), start (bp), end (bp).

Parameters:

- **CNV_dict** - dict. Gene name (key) and whether they should be 'gain' or 'loss'. Could for example be gain of known oncogenes or loss of tumor supressors. Gene selection inspo: https://doi.org/10.1080%2F07853890.2023.2280708
- **min_size** - nbr in bp.
- **max_size** - nbr in bp. A size of the CNV is generated as a random number between the min and max
- **gene_info** - the *Ensmbl_BioMart_gene_info.txt* containing 'Gene stable ID', 'Chromosome/scaffold name', 'Gene start (bp)', 'Gene end (bp)', 'Gene name'

Returns:

- **CNV_df** (DataFrame): Compiling the gene name, chromosome, size, type, start, end

In [10]:
adata.var

,feature_is_filtered,feature_name,feature_reference,feature_biotype,feature_length,feature_type,n_cells_by_counts,mean_counts,log1p_mean_counts,pct_dropout_by_counts,total_counts,log1p_total_counts,chromosome,start,end
ENSG00000238009,False,ENSG00000238009.6,NCBITaxon:9606,gene,629,lncRNA,40,0.002678,0.002675,99.353169,16.561602,2.865715,chr1,89295.0,133566.0
ENSG00000239945,False,ENSG00000239945.1,NCBITaxon:9606,gene,1319,lncRNA,4,0.000240,0.000240,99.935317,1.482688,0.909342,chr1,89551.0,91105.0
ENSG00000237491,False,LINC01409,NCBITaxon:9606,gene,1059,lncRNA,391,0.030694,0.030233,93.677232,189.814697,5.251303,chr1,778747.0,810065.0
ENSG00000177757,False,FAM87B,NCBITaxon:9606,gene,1947,lncRNA,44,0.003231,0.003226,99.288486,19.980288,3.043583,chr1,817371.0,819837.0
ENSG00000225880,False,LINC00115,NCBITaxon:9606,gene,3312,lncRNA,167,0.013537,0.013446,97.299483,83.714722,4.439290,chr1,824228.0,827539.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000278817,False,ENSG00000278817.1,NCBITaxon:9606,gene,1213,protein_coding,351,0.025161,0.024850,94.324062,155.596588,5.053673,NaN,NaN,NaN
ENSG00000277196,False,ENSG00000277196.4,NCBITaxon:9606,gene,2197,protein_coding,2,0.000107,0.000107,99.967658,0.664546,0.509553,NaN,NaN,NaN
ENSG00000278384,False,ENSG00000278384.1,NCBITaxon:9606,gene,3027,protein_coding,63,0.004674,0.004663,98.981242,28.904743,3.398017,NaN,NaN,NaN
ENSG00000276345,False,ENSG00000276345.1,NCBITaxon:9606,gene,740,protein_coding,1746,0.147193,0.137319,71.765847,910.245239,6.814812,NaN,NaN,NaN


In [13]:
def gene_exists(adata, gene_list):
    for gene in gene_list:
        if gene in adata.var['feature_name']:
            print(gene, 'exists! \n')
        else:
            print(gene, 'does not exist.. \n')

In [17]:
gene_exists(adata, gene_list=['DIS3','MECOM','ERBB2','CHD7','HCK','KEAP1','MYD88','TBX3']) #'EPHB1','FLT1',

# CNVs, including ABCC5, AGO2, ARID5B, CHD7, FAM58A, FOXA1, HEY1, HLA-C, HLA-DQB1, MCL1, MECOM, MSN, NFKBIA, PRSS1, RAD21,

DIS3 does not exist.. 

MECOM does not exist.. 

ERBB2 does not exist.. 

CHD7 does not exist.. 

HCK does not exist.. 

KEAP1 does not exist.. 

MYD88 does not exist.. 

TBX3 does not exist.. 



In [48]:
CNV_dict = { 
    'DIS3': 'loss',
    'MECOM': 'loss',
    'ERBB2': 'gain',
    'CHD7': 'gain',
    'HCK': 'gain',
    'KEAP1': 'loss',
    'MYD88': 'gain',
    'TBX3': 'gain'
}

min_size = 1000000
max_size = 5000000


gene_file = ("/home/augusta/SSS_mount/insituCNV/InSituCNV/Ensmbl_BioMart_gene_info.txt")

In [49]:
# CNV_df = generate_cnvs(CNV_dict, min_size, max_size, gene_info=gene_file, save_csv='CNVs_121224.csv') 
CNV_df

,Gene name,Chromosome,Size (bp),Type,Start (bp),End (bp)
0,DIS3,13,3751983,loss,70891141,74643123
1,MECOM,3,1804900,loss,168471187,170276087
2,ERBB2,17,3809612,gain,37804364,41613976
3,CHD7,8,1706602,gain,59920083,61626685
4,HCK,20,1035499,gain,31559277,32594775
5,KEAP1,19,3753799,loss,8617942,12371740
6,MYD88,3,1859257,gain,37211160,39070416
7,TBX3,12,4773515,gain,112290458,117063972


#### ...or extract CNVs

In [10]:
CNV_df = pd.read_csv("/home/augusta/SSS_mount/insituCNV/InSituCNV/Figure2/01_Simulate_CNVs_in_spatial_data/Simulate_CNVs_vascular_normal/CNVs_121224.csv")

In [11]:
CNV_df

,Gene name,Chromosome,Size (bp),Type,Start (bp),End (bp)
0,DIS3,13,3751983,loss,70891141,74643123
1,MECOM,3,1804900,loss,168471187,170276087
2,ERBB2,17,3809612,gain,37804364,41613976
3,CHD7,8,1706602,gain,59920083,61626685
4,HCK,20,1035499,gain,31559277,32594775
5,KEAP1,19,3753799,loss,8617942,12371740
6,MYD88,3,1859257,gain,37211160,39070416
7,TBX3,12,4773515,gain,112290458,117063972


### Function: Create CNV template (create_cnv_template)

This function generates a CNV template for an AnnData object based on a predifined number of subclones, and their CNVs, generated by the generate_cnvs function.  

Parameters:

- **adata** - the AnnData object
- **CNV_df** (DataFrame): from the *'Generate CNVs'* module. the gene name, chromosome, size, type, start, end - . The type should generate either -1/+1 depending on if it's 'gain' or 'loss'.

Returns:

- **cnv_template_df** (DataFrame). For each CNV (gene name) - generate an array with -1/0/1 values for each gene in the adata.var_names

In [17]:
cnv_template_df = create_cnv_template(adata, CNV_df)
cnv_template_df

gene_ids,ENSG00000238009,ENSG00000241860,ENSG00000241599,ENSG00000286448,ENSG00000237491,ENSG00000225880,ENSG00000228794,ENSG00000230368,ENSG00000272438,ENSG00000223764,...,ENSG00000287171,ENSG00000196664,ENSG00000228933,ENSG00000224294,ENSG00000122824,ENSG00000158639,ENSG00000269437,ENSG00000204025,ENSG00000165509,ENSG00000126895
DIS3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
MECOM,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ERBB2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CHD7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
HCK,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
KEAP1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
MYD88,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TBX3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Function: Simualte CNVs in the data (simulate_cnvs)

This function applies poission distribution / binomial probability to simulate one arm gains and losses (increase/keep/decrease the counts (adata.X) depending on the -1/0/1 value of the cnv_template_df: if -1, the probability is 0.5 and if 1 it is 1.5). This is done according to a subclone_dict, where each key is a subclone, and the values state which CNVs to add to these cells.  

Parameters:

- **adata** - the AnnData object to add the CNVs on
- **cnv_template_df** matrix. from the *'Create CNV template'* module
- **subclone_dict** (dictionary) For each subclone (key), a list of which CNVs to assign (values). randomly assigning each cell to one of these subclones (adata.obs.simulated_subclone)

Returns:

- **adata** - the AnnData object with simulated CNVs as a layer to the adata.X, (as well as the CNV template as a 'CNV_template' layer?)

In [12]:
adata

AnnData object with n_obs × n_vars = 19311 × 22798
    obs: 'mapped_reference_annotation', 'donor_id', 'self_reported_ethnicity_ontology_term_id', 'donor_living_at_sample_collection', 'organism_ontology_term_id', 'sample_uuid', 'sample_preservation_method', 'tissue_ontology_term_id', 'development_stage_ontology_term_id', 'sample_derivation_process', 'donor_BMI_at_collection', 'tissue_type', 'suspension_derivation_process', 'suspension_dissociation_reagent', 'suspension_dissociation_time', 'suspension_uuid', 'suspension_type', 'library_uuid', 'assay_ontology_term_id', 'sequencing_platform', 'is_primary_data', 'cell_type_ontology_term_id', 'author_cell_type', 'disease_ontology_term_id', 'sex_ontology_term_id', 'nCount_RNA', 'nFeature_RNA', 'cell_type', 'assay', 'disease', 'organism', 'sex', 'tissue', 'self_reported_ethnicity', 'development_stage', 'observation_joinid', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct

In [12]:
subclone_dict = {
    'N': [], # Normal cells without CNV simulations
    'A': ['DIS3', 'MECOM', 'ERBB2', ], # Original genetic subclone
    'B': ['DIS3', 'MECOM', 'ERBB2', 'CHD7', 'HCK', 'KEAP1'], 
    'C': ['DIS3', 'MECOM', 'ERBB2', 'CHD7', 'MYD88', 'TBX3'],
} # 'DIS3','MECOM','ERBB2','CHD7','HCK','KEAP1','MYD88','TBX3'

In [13]:
# sc.pp.normalize_total(adata, layer='counts')
# sc.pp.log1p(adata, layer='counts')

In [14]:
print(adata.layers['counts'])

<Compressed Sparse Row sparse matrix of dtype 'float32'
	with 66535522 stored elements and shape (8892, 25691)>
  Coords	Values
  (0, 12)	1.0
  (0, 16)	2.0
  (0, 17)	1.0
  (0, 18)	1.0
  (0, 26)	2.0
  (0, 29)	1.0
  (0, 32)	1.0
  (0, 34)	1.0
  (0, 39)	7.0
  (0, 42)	4.0
  (0, 48)	1.0
  (0, 51)	2.0
  (0, 58)	1.0
  (0, 64)	9.0
  (0, 66)	1.0
  (0, 70)	1.0
  (0, 77)	1.0
  (0, 79)	3.0
  (0, 80)	1.0
  (0, 86)	1.0
  (0, 89)	1.0
  (0, 109)	20.0
  (0, 111)	1.0
  (0, 112)	2.0
  (0, 121)	2.0
  :	:
  (8891, 23224)	1.0
  (8891, 23335)	0.0
  (8891, 23487)	1.0
  (8891, 23547)	3.0
  (8891, 23593)	1.0
  (8891, 23611)	1.0
  (8891, 23624)	4.0
  (8891, 23776)	12.0
  (8891, 23856)	1.0
  (8891, 24003)	1.0
  (8891, 24045)	2.0
  (8891, 24213)	1.0
  (8891, 24317)	1.0
  (8891, 24861)	1.0
  (8891, 24867)	2.0
  (8891, 24943)	2.0
  (8891, 24969)	1.0
  (8891, 25018)	1.0
  (8891, 25023)	2.0
  (8891, 25105)	2.0
  (8891, 25185)	1.0
  (8891, 25311)	2.0
  (8891, 25398)	1.0
  (8891, 25582)	1.0
  (8891, 25601)	1.0


In [15]:
adata.obs.cell_type.unique()

['secretory cell', 'basal cell', 'ciliated cell']
Categories (3, object): ['ciliated cell', 'secretory cell', 'basal cell']

In [ ]:
simulate_cnvs(adata, cnv_template_df, subclone_dict, cell_type_reference='cell_type', cell_type_cnv='basal cell', alpha=6) #, alpha=6

In [ ]:
adata.write("lung_organoids_simulatedCNVs_121224_simulationv2_rho6", compression = 'gzip')